In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %aimport data_processing
# %aimport codelists
# %aimport constants
# %aimport import_file

In [3]:
from data_processing import *

In [4]:
from import_file import *

In [5]:
pd.options.display.max_rows = 200

In [6]:
# create_pegmed()

In [7]:
# create_pegprod()

In [8]:
# create_medcoded_entries()

In [9]:
# create_prescriptions()

In [10]:
# create_consultations()

In [11]:
# all_entries = create_all_entries()

In [12]:
# all_entries = pd.read_csv('data/pt_data/all_entries.csv',delimiter=',',parse_dates=['sysdate','eventdate'],infer_datetime_format=True)

In [13]:
all_entries = pd.read_hdf('hdf/all_entries.hdf')

In [14]:
all_entries.sample(5)

,eventdate,medcode,patid,prodcode,sysdate,type
72212132,2007-07-20,74.0,3215672,NaN,2007-07-20,2
117352821,2004-07-12,NaN,5106172,823.0,2004-07-12,5
45895102,2008-04-17,461.0,14864038,NaN,2008-04-17,2
125284930,2003-06-23,NaN,10117250,7.0,2003-06-23,5
120101235,2011-08-31,NaN,7945199,5828.0,2011-08-31,5


In [15]:
# all_entries.to_hdf('hdf/all_entries.hdf', 'all_entries' ,mode='w')

In [16]:
create_pt_features()

In [17]:
get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries)

In [18]:
add_data_start_and_end_dates(all_entries)

In [19]:
calculate_amount_of_data_available(isCase=True)

In [20]:
delete_patients_if_not_enough_data(isCase=True)

In [21]:
match_cases_and_controls()

In [22]:
delete_unmatched_cases_and_controls()

In [23]:
calculate_amount_of_data_available(isCase=False)

In [24]:
get_insomnia_event_count(entries=all_entries)

In [25]:
pt_features = pd.read_csv('data/pt_data/pt_features.csv',delimiter=',')

In [26]:
len(pt_features)

8414

In [27]:
pt_features.sort_values(by='matchid')

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,days pre_indexdate,days post_indexdate,matchid,insom_count
5890,5898001,2,1935,1,2008-08-12,False,NaN,1997-03-07,2013-05-14,4176.0,1736.0,5,0
5903,644001,2,1935,1,2008-08-12,True,1350.0,1997-03-07,2013-05-17,4176.0,1739.0,5,0
3809,803001,1,1923,1,2007-09-12,True,1350.0,1997-03-07,2012-01-19,3841.0,1590.0,10,0
3760,5551001,1,1923,1,2007-09-12,False,NaN,1997-03-07,2012-01-04,3841.0,1575.0,10,0
5831,1064001,2,1928,1,2011-03-03,True,1917.0,1997-03-07,2013-05-02,5109.0,791.0,16,0
5796,2192001,2,1928,1,2011-03-03,False,NaN,1997-03-07,2013-04-25,5109.0,784.0,16,0
5778,1645001,1,1929,1,2007-05-24,True,26270.0,1997-03-07,2013-04-23,3730.0,2161.0,18,0
4841,5988001,1,1929,1,2007-05-24,False,NaN,1997-03-07,2012-10-26,3730.0,1982.0,18,0
5898,2136001,2,1923,1,2009-03-16,True,6578.0,1997-03-07,2013-05-16,4392.0,1522.0,22,0
2763,9687001,2,1923,1,2009-03-16,False,NaN,1997-03-07,2011-05-05,4392.0,780.0,22,0
